# Project 2: Đồ án Gram – Schmidt
#### Họ và tên: Trần Anh khoa
#### Lớp: 23CTT2
#### Mã số sinh viên: 23120135

## Yêu cầu 1: 
Cho A là ma trận có thể phân rã QR. Sinh viên viết chương trình in ra ma trận Q và R, biết rằng A = QR.


- Nền tảng để giải quyết vấn đề là thông qua giải thuật Gram-Schmidt.
- Giải thuật Gram-Schmidt dùng để trực giao hóa một tập hợp các vector độc lập tuyến tính trong không gian Euclid. Chi tiết về giải thuật Gram-Schmidt được trình bày theo các bước như sau: 

## Giải thuật Gram-Schmidt

Giả sử ta có tập $S$ gồm các vector độc lập tuyến tính như sau: $S = \{u_1, u_2, u_3, u_4, \dots, u_k\}$.
Lần lượt thực hiện k bước sau:

**Bước 1**. Đặt $ v_1 = u_1 $.

**Bước 2**. Đặt $ v_2 = u_2 - \frac{\langle u_2, v_1 \rangle}{\|v_1\|^2} v_1 $.

**Bước 3**. Đặt $ v_3 = u_3 - \frac{\langle u_3, v_1 \rangle}{\|v_1\|^2} v_1 - \frac{\langle u_3, v_2 \rangle}{\|v_2\|^2} v_2 $.

**Bước 4**. Đặt $ v_4 = u_4 - \frac{\langle u_4, v_1 \rangle}{\|v_1\|^2} v_1 - \frac{\langle u_4, v_2 \rangle}{\|v_2\|^2} v_2 - \frac{\langle u_4, v_3 \rangle}{\|v_3\|^2} v_3 $.

...

Cho tới bước k.
- Khi đó ta thu được tập $\{v_1, v_2, v_3, v_4, \dots, v_k\}$ chứa các vector vuông góc với nhau từng đôi một (hệ trực giao).
- Ngoài ra, ta còn xây dựng được hệ trực chuẩn $\{q_1, q_2, \dots, q_n\}$ từ hệ $S$ bằng cách đặt:

$$
q_i = \frac{v_i}{\|v_i\|}, \quad \text{với} \quad i = 1, 2, \dots, n.
$$

Sau đây là mã nguồn thực hiện yêu cầu đề bài.

In [15]:
# Hàm trích xuất cột của một ma trận
def extract_column(matrix, col_index):
    return [row[col_index] for row in matrix]

# Hàm tạo ma trận có kích thước row x col mà các số hạng đều là số 0
def create_zero_matrix(row, col):
    return [[0 for _ in range(col)] for _ in range(row)]

# Hàm tính chuẩn (norm) của vector
def norm(vec):
    result = 0
    for v in vec:
        result += v * v
    return result ** 0.5

# Hàm nhân một số với vector
def multiply_by_number(number, vector):
    result = []
    for element in vector:
        result.append(number * element)
    return result

# Hàm trừ hai vector
def subtract_vectors(vec1, vec2):
    if len(vec1) != len(vec2):
        raise ValueError("Hai vector cần có cùng độ dài")
    
    result = []
    for i in range(len(vec1)):
        result.append(vec1[i] - vec2[i])
    return result
    
# Hàm tính tích vô hướng của hai vector
def dot_product_vectors(vec1, vec2):
    if len(vec1) != len(vec2):
        raise ValueError("Hai vector phải có cùng độ dài")
    
    result = 0
    for i in range(len(vec1)):
        result += vec1[i] * vec2[i]
    return result

# Hàm nhân hai ma trận
def multiply_matrix(A_list, B_list):
    result_list = [[0 for _ in range(len(B_list[0]))] for _ in range(len(A_list))]

    m_row_A = len(A_list)
    n_col_B = len(B_list[0])
    for i_row in range(m_row_A):
        for i_col in range(n_col_B):
            total = 0
            for i,a in enumerate(A_list[i_row]): 
                total += a*B_list[i][i_col]
            result_list[i_row][i_col] = total

    return result_list

# Hàm chuyển vị ma trận
def transpose(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    result_matrix = create_zero_matrix(cols, rows)
    for i in range(rows):
        for j in range(cols):
            result_matrix[j][i] = matrix[i][j]
    return result_matrix

# Hàm in ma trận
def print_matrix(matrix, name="Matrix"):
    print(f"{name}:")
    for row in matrix:
        print([round(val, 8) if isinstance(val, float) else val for val in row])
    print()

# Hàm chuẩn hóa ma trận để loại bỏ -0.0 và các số rất nhỏ
def normalize_matrix(matrix):
    row = len(matrix)
    col = len(matrix[0])
    
    for i in range(row):
        for j in range(col):
            # Kiểm tra số rất nhỏ 
            if abs(matrix[i][j]) < 1e-9:
                matrix[i][j] = 0.0
            # Kiểm tra và thay đổi các số -0.0
            if matrix[i][j] == -0.0:
                matrix[i][j] = 0.0
    
    return matrix

# Hàm phân rã QR 
def qr_decomposition(A):
    m = len(A)  
    n = len(A[0]) 

    # Khởi tạo ma trận Q và ma trận R
    Q = create_zero_matrix(m, n) 
    R = create_zero_matrix(n, n) 

    # List các cột của ma trận A
    a_vectors = [extract_column(A, j) for j in range(n)]
    
    # Khởi tạo list lưu trữ vector trực chuẩn e_i
    e_vectors = []  

# Quá trình thực hiện thuật toán Gram-Schmidt
    
    for i in range(n):
        # Bước lấy f_i
        if i == 0:
            # f_1 chính là a_1
            f_i = a_vectors[i]
        else:
            # Với i >= 1, f_i được tính từ a_i và các e_j trước đó
            f_i = a_vectors[i]
            
            for j in range(i):
                proj = dot_product_vectors(a_vectors[i], e_vectors[j])
                f_i = subtract_vectors(f_i, multiply_by_number(proj, e_vectors[j]))
        
        # Bước 2: Tính e_i từ f_i
        norm_fi = norm(f_i)
        if norm_fi == 0:
            raise ValueError("Các cột của ma trận không độc lập tuyến tính")
        e_i = multiply_by_number(1 / norm_fi, f_i)
        e_vectors.append(e_i)

        # Cập nhật cột i của ma trận Q
        for row in range(m):
            Q[row][i] = e_i[row]

    # Xây dựng ma trận R
    for i in range(n):
        for j in range(i, n):
            R[i][j] = dot_product_vectors(a_vectors[j], e_vectors[i])

    return Q, R

# Hàm kiểm tra hai ma trận có bằng nhau không (với ngưỡng sai số cho phép)
def check_matrix_similarity(matrix1, matrix2, e = 1e-9):
    if len(matrix1) != len(matrix2) or len(matrix1[0]) != len(matrix2[0]):
        return False
    for i in range(len(matrix1)):
        for j in range(len(matrix1[0])):
            if abs(matrix1[i][j] - matrix2[i][j]) > e:
                return False
    return True

# Ma trận A để phân rã QR
A = [
    [4, 1, 2, 2],
    [1, 3, 3, 1],
    [2, 3, 4, 3],
    [2, 1, 3, 4]
]
    
print_matrix(A, "Ma trận A")
    
# Phân rã QR
Q, R = qr_decomposition(A)
    
print_matrix(Q, "Ma trận Q")
print_matrix(R, "Ma trận R")
    
# Kiểm tra: A = Q * R
A_reconstructed = normalize_matrix(multiply_matrix(Q, R)) #Cần chuẩn hóa lại ma trận để loại bỏ các giá trị rất nhỏ và giá trị -0.0
print_matrix(A_reconstructed, "Ma trận A_reconstructed được tái tạo từ việc lấy Q * R")

# Sử dụng hàm đã xây dựng ở trên để kiểm tra xem ma trận A ban đầu và ma trận A được tái tạo có bằng nhau không?
equal = check_matrix_similarity(A, A_reconstructed)
if equal:
    print("Ma trận A và A_reconstructed bằng nhau (với mức sai số cho phép).")
else:
    print("Ma trận A và A_reconstructed không bằng nhau.")

Ma trận A:
[4, 1, 2, 2]
[1, 3, 3, 1]
[2, 3, 4, 3]
[2, 1, 3, 4]

Ma trận Q:
[0.8, -0.42211588, -0.42426407, -0.04264014]
[0.2, 0.72362723, -0.28284271, -0.59696201]
[0.4, 0.54272042, 0.14142136, 0.72488244]
[0.4, -0.06030227, 0.84852814, -0.34112115]

Ma trận R:
[5.0, 3.0, 5.0, 4.6]
[0, 3.31662479, 3.31662479, 1.26634765]
[0, 0, 1.41421356, 2.68700577]
[0, 0, 0, 0.12792043]

Ma trận A_reconstructed được tái tạo từ việc lấy Q * R:
[4.0, 1.0, 2.0, 2.0]
[1.0, 3.0, 3.0, 1.0]
[2.0, 3.0, 4.0, 3.0]
[2.0, 1.0, 3.0, 4.0]

Ma trận A và A_reconstructed bằng nhau (với mức sai số cho phép).


## Mô tả ý tưởng
Giả sử ta có một ma trận $A$ có kích thước $m \times n$ được biểu diễn ở dạng $A = [ a_1, a_2, a_3, \dots , a_n ]$ với $S = \{a_1, a_2, a_3, \dots , a_n\}$ là tập các vector cột tạo nên ma trận $A$.

Khi đó, giải thuật Gram-Schmidt sẽ giúp ta chuyển tập $S$ thành tập các vector trực giao $\{f_1, f_2, f_3, \dots , f_n\}$. Và ta cũng có thể tìm tập các vector trực chuẩn tương ứng $\{e_1, e_2, e_3, \dots , e_n\}$ với:

$$
q_i = \frac{v_i}{\|v_i\|}, \quad \text{với} \quad i = 1, 2, \dots, n.
$$

Cụ thể các bước như sau:
- **Bước 1**: Đặt $f_1 = a_1$. Khi đó $e_1 = \frac{f_1}{\|f_1\|}$.
- **Bước 2**: Tính $f_2 = a_2 - \langle a_2, e_1 \rangle \cdot e_1$. Khi đó $e_2 = \frac{f_2}{\|f_2\|}$.
- **Bước 3**: Tính $f_3 = a_3 - \langle a_3, e_1 \rangle \cdot e_1 - \langle a_3, e_2 \rangle \cdot e_2$. Khi đó $e_3 = \frac{f_3}{\|f_3\|}$.
- ...
- **Bước n**: Tính $f_n = a_n - \langle a_n, e_1 \rangle \cdot e_1 - \langle a_n, e_2 \rangle \cdot e_2 - \dots - \langle a_n, e_{n-1} \rangle \cdot e_{n-1}$. Khi đó $e_n = \frac{f_n}{\|f_n\|}$.

Khi đó ta xây dựng được ma trận $Q = [e_1, e_2, e_3, \dots, e_n]$ có kích thước $m \times n$ với tập $\{e_1, e_2, e_3, \dots, e_n\}$ là tập các vector cột. Đồng thời ta cũng xây dựng được ma trận tam giác vuông $R$ có kích thước $n \times n$ như sau:
$$ 
R = 
\begin{pmatrix}
\|f_1\| & \langle a_2, e_1 \rangle & \langle a_3, e_1 \rangle & \dots & \langle a_n, e_1 \rangle \\
0 & \|f_2\| & \langle a_3, e_2 \rangle & \dots & \langle a_n, e_2 \rangle \\
0 & 0 & \|f_3\| & \dots & \langle a_n, e_3 \rangle \\
0 & 0 & 0 & \dots & \langle a_n, e_4 \rangle \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \dots & \|f_n\|
\end{pmatrix}
$$
Lúc này hai ma trận Q và R thỏa mãn: A = QR theo yêu cầu đề bài.

## Mô tả các hàm

### 1. Hàm `extract_column(matrix, col_index)`
- **Tác dụng**: Trích xuất vector cột của ma trận.
- **Input**:
  - `matrix`: Ma trận đầu vào.
  - `col_index`: Chỉ số cột mà người dùng muốn trích xuất.
- **Nguyên lý hoạt động**:
  - Sử dụng list comprehension để duyệt qua từng hàng của ma trận.
  - Lấy phần tử tại chỉ số cột `col_index` của mỗi hàng (`row[col_index]`).
  - Tạo một danh sách chứa các phần tử của cột được trích xuất.
- **Output**: Danh sách chứa các phần tử của vector cột.

---

### 2. Hàm `create_zero_matrix(row, col)`
- **Tác dụng**: Tạo ma trận có kích thước `row x col` với tất cả các số hạng đều bằng 0.
- **Input**:
  - `row`: Số hàng của ma trận.
  - `col`: Số cột của ma trận.
- **Nguyên lý hoạt động**:
  - Sử dụng list comprehension lồng nhau:
    - Vòng ngoài tạo `row` hàng.
    - Vòng trong tạo mỗi hàng với `col` số hạng có giá trị 0.
- **Output**: Ma trận kích thước `row x col` với tất cả phần tử bằng 0.

---

### 3. Hàm `norm(vec)`
- **Tác dụng**: Tính chuẩn của một vector.
- **Input**: Một vector.
- **Nguyên lý hoạt động**:
  - Khởi tạo biến `result = 0` để lưu tổng bình phương các phần tử.
  - Duyệt qua từng phần tử trong `vec` và cộng bình phương của mỗi phần tử vào `result`.
- **Output**: Hàm trả về căn bậc 2 của `result`, tức là chuẩn của vector.

---

### 4. Hàm `multiply_by_number(number, vector)`
- **Tác dụng**: Nhân một số với vector.
- **Input**:
  - `number`: Số thực hoặc số nguyên để nhân.
  - `vector`: Vector cần nhân với `number`.
- **Nguyên lý hoạt động**:
  - Khởi tạo danh sách `result` (rỗng) để lưu kết quả.
  - Duyệt qua từng phần tử trong vector, nhân phần tử này với `number` và thêm kết quả vào `result`.
- **Output**: Danh sách `result` chứa kết quả.

---

### 5. Hàm `subtract_vectors(vec1, vec2)`
- **Tác dụng**: Tính phép trừ hai vector.
- **Input**:
  - `vec1`: Vector thứ nhất.
  - `vec2`: Vector thứ hai.
- **Nguyên lý hoạt động**:
  - Kiểm tra xem hai vector có cùng độ dài không:
    - Nếu không, in ra lỗi kèm thông báo.
  - Khởi tạo danh sách `result` (rỗng) để lưu kết quả.
  - Duyệt qua các chỉ số `i` từ 0 đến độ dài vector và tính `vec1[i] - vec2[i]`, lưu vào `result`.
- **Output**: Danh sách `result` chứa kết quả.

---

### 6. Hàm `dot_product_vectors(vec1, vec2)`
- **Tác dụng**: Tính tích vô hướng của hai vector.
- **Input**:
  - `vec1`: Vector thứ nhất.
  - `vec2`: Vector thứ hai.
- **Nguyên lý hoạt động**:
  - Kiểm tra xem hai vector có cùng độ dài không:
    - Nếu không, in ra lỗi kèm thông báo.
  - Khởi tạo biến `result = 0` để lưu kết quả.
  - Duyệt qua các chỉ số `i` từ 0 đến độ dài vector và tính `vec1[i] * vec2[i]`, cộng vào `result`.
- **Output**: `result`, một số (thường là số thực) là tích vô hướng của hai vector.

---

### 7. Hàm `multiply_matrix(A_list, B_list)`
- **Tác dụng**: Tính phép nhân hai ma trận.
- **Input**:
  - `A_list`: Ma trận A, kích thước `m x p`.
  - `B_list`: Ma trận B, kích thước `p x n`.
- **Nguyên lý hoạt động**:
  - Tạo ma trận kết quả `result_list` với kích thước `m x n`, ban đầu tất cả phần tử bằng 0.
  - Duyệt qua từng hàng `i_row` của A và từng cột `i_col` của B:
    - Tính phần tử `result_list[i_row][i_col]` bằng cách cộng tích của các phần tử tương ứng từ `A` và `B`.
- **Output**: Ma trận kích thước `m x n` là kết quả của phép nhân `A x B`.

---

### 8. Hàm `transpose(matrix)`
- **Tác dụng**: Chuyển vị ma trận.
- **Input**:
  - `matrix`: Ma trận đầu vào kích thước `m x n`.
- **Nguyên lý hoạt động**:
  - Lấy số hàng và số cột của ma trận đầu vào.
  - Tạo ma trận kết quả `result_matrix` với kích thước `n x m`.
  - Duyệt qua từng hàng `i` và cột `j` của ma trận gốc và hoán đổi chỉ số hàng và cột của ma trận ban đầu.
- **Output**: Ma trận kích thước `n x m` là ma trận chuyển vị của ma trận ban đầu.

---

### 9. Hàm `print_matrix(matrix, name="Matrix")`
- **Tác dụng**: In ra ma trận.
- **Input**:
  - `matrix`: Ma trận cần in.
  - `name`: Tên của ma trận (mặc định là "Matrix").
- **Nguyên lý hoạt động**:
  - In tên ma trận.
  - Duyệt qua từng hàng `row` trong ma trận và xử lý từng phần tử:
    - Nếu là số thực, làm tròn đến 8 chữ số thập phân.
    - Nếu không, giữ nguyên giá trị.
  - In mỗi hàng đã xử lý.
- **Output**: Không có, chỉ thực hiện in ma trận ra màn hình.

---

### 10. Hàm `normalize_matrix(matrix)`
- **Tác dụng**: Chuẩn hóa ma trận bằng cách loại bỏ các giá trị rất nhỏ (gần 0) và xử lý các giá trị -0.0.
- **Input**:
  - `matrix`: Ma trận cần chuẩn hóa.
- **Nguyên lý hoạt động**:
  - Lấy số hàng và số cột của ma trận.
  - Duyệt qua từng số hạng:
    - Nếu giá trị nhỏ gần 0, gán về 0.0.
    - Nếu là -0.0, cũng gán về 0.0.
- **Output**: Ma trận đã được chuẩn hóa.

---

### 11. Hàm `qr_decomposition(A)`
- **Tác dụng**: Thực hiện phân rã QR của ma trận A.
- **Input**: Ma trận A kích thước `m x n`.
- **Nguyên lý hoạt động**:
  - Lấy số dòng và số cột của ma trận A.
  - Khởi tạo ma trận `Q` và ma trận `R`.
  - Dùng thuật toán Gram-Schmidt để tính các vector trực giao `f_i` và trực chuẩn `e_i`.
  - Xây dựng ma trận `R` từ tích vô hướng của các vector.
- **Output**: Ma trận `Q` và ma trận `R` sao cho \( A = QR \).

----
### 12. Hàm `check_matrix_similarity(A, A_reconstructed)`
- **Tác dụng**: Thực hiện việc kiểm tra hai ma trận có bằng nhau hay không với mức sai số cho phép.
- **Input**: Hai ma trận cần kiểm tra.
- **Nguyên lý hoạt động**:
  - Kiểm tra xem nếu số dòng và số cột của hai ma trận khác nhau thì trả về false.
  - Kiểm tra từng giá trị tương ứng của hai ma trận có bằng nhau hay không với mức sai số cho phép.
    + Nếu phát hiện một giá trị bất kì khác nhau (chênh lệch lớn hơn mức sai số cho phép) thì trả về false.
  - Nếu không vi phạm những điều kiện trên thì trả về true.
- **Output**: True/False

## Yêu cầu 2:
### Kiểm tra kết quả với hàm có sẵn
Trong phần này ta sẽ thực hiện việc kiểm tra kết quả của hàm phân rã QR do tôi tự xây dựng ở Yêu cầu 1 với hàm có sẵn trong thư viện của Python. Sau đây là đoạn mã nguồn sử dụng thư viện **NumPy** và một hàm trong thư viện đó là **np.linalg.qr()** để thực hiện phân rã QR.

In [16]:
import numpy as np

A = np.array([
    [4, 1, 2, 2],
    [1, 3, 3, 1],
    [2, 3, 4, 3],
    [2, 1, 3, 4]
])

Q, R = np.linalg.qr(A)

print("Ma trận A")
print(A)
print("\nMa trận Q")
print(Q)
print("\nMa trận R")
print(R)

Ma trận A
[[4 1 2 2]
 [1 3 3 1]
 [2 3 4 3]
 [2 1 3 4]]

Ma trận Q
[[-0.8         0.42211588  0.42426407  0.04264014]
 [-0.2        -0.72362723  0.28284271  0.59696201]
 [-0.4        -0.54272042 -0.14142136 -0.72488244]
 [-0.4         0.06030227 -0.84852814  0.34112115]]

Ma trận R
[[-5.         -3.         -5.         -4.6       ]
 [ 0.         -3.31662479 -3.31662479 -1.26634765]
 [ 0.          0.         -1.41421356 -2.68700577]
 [ 0.          0.          0.         -0.12792043]]


---
Ta nhận thấy rằng, với cùng một ma trận **A**, kết quả của việc phân rã QR được thực hiện bởi chương trình tự xây dựng của tôi và hàm có sẵn trong Python là giống nhau. (Sự khác biệt về **dấu** trong các ma trận **Q** và **R** là điều bình thường và không làm thay đổi tính chính xác của phân rã QR. Điều này **không ảnh hưởng** đến việc tái tạo lại ma trận **A** khi thực hiện phép nhân **Q** $\times$ **R**).


### Ứng dụng của phân rã QR

#### **1. Giải hệ phương trình tuyến tính**

##### **Mô tả tổng quát**:
Giải hệ phương trình tuyến tính dạng $Ax = b$ là một trong những ứng dụng cơ bản của phân rã QR. Trong đó, $A$ là ma trận hệ số và $x$ là vector ẩn cần tìm. Phân rã QR phân tách ma trận $A$ thành hai ma trận: $A = QR$, với $Q$ là ma trận trực giao và $R$ là ma trận tam giác trên.

##### **Cách áp dụng**:
Khi ta phân rã $A = QR$, hệ phương trình $Ax = b$ trở thành:
$$
QRx = b
$$
Do $Q$ là ma trận trực giao (có $Q^T Q = I$), ta nhân cả hai vế của phương trình với $Q^T$ (ma trận chuyển vị của $Q$):
$$
R x = Q^T b
$$
Lúc này, bài toán trở thành giải phương trình tam giác trên $R x = Q^T b$, dễ dàng thực hiện bằng phương pháp thay ngược (back substitution).

##### **Ứng dụng thực tế**:
- **Hồi quy tuyến tính**: Phân rã QR giúp giải bài toán tối thiểu hóa sai số trong hồi quy tuyến tính, đặc biệt khi ma trận $A$ có nhiều đặc trưng hoặc khi dữ liệu có chiều cao.
- **Tối ưu hóa**: Giải quyết các bài toán tối ưu liên quan đến các phương trình tuyến tính, đặc biệt trong các hệ thống lớn hoặc khi không thể giải trực tiếp.

Phân rã QR mang lại sự ổn định số học cao hơn so với các phương pháp khác (như phân rã LU), giúp tránh được các vấn đề về sai số số học khi giải các hệ phương trình tuyến tính với ma trận gần suy biến.

---

#### **2. Tính giá trị riêng và vector riêng**

##### **Mô tả tổng quát**:
Phân rã QR là nền tảng của **thuật toán QR**, phương pháp lặp để tính giá trị riêng (eigenvalue) và vector riêng (eigenvector) của một ma trận vuông $A$. Phương pháp này đặc biệt quan trọng trong các bài toán phân tích dữ liệu và mô phỏng vật lý.

##### **Cách áp dụng**:
Bắt đầu với ma trận $A$, ta thực hiện phân rã QR:
$$
A_k = Q_k R_k
$$
Sau đó, ta cập nhật ma trận $A_{k+1} = R_k Q_k$. Qua nhiều lần lặp, ma trận $A_k$ sẽ dần dần hội tụ về dạng tam giác (hoặc gần tam giác), và các giá trị riêng sẽ xuất hiện trên đường chéo của ma trận tam giác cuối cùng.

##### **Ứng dụng thực tế**:
- **Phân tích dữ liệu**: Tìm các thành phần chính trong phân tích thành phần chính (PCA), giúp giảm chiều dữ liệu và phát hiện các mẫu quan trọng.
- **Mô phỏng vật lý**: Tính toán các giá trị riêng và vector riêng trong các mô hình cơ học hoặc điện từ học, giúp hiểu rõ hơn về hành vi của các hệ thống vật lý.

Thuật toán QR ổn định và có thể tính được giá trị và vector riêng của ma trận mà không gặp phải vấn đề về sai số số học như các phương pháp khác.

---

#### **3. Hồi quy tuyến tính và bài toán bình phương tối thiểu**

##### **Mô tả tổng quát**:
Trong thống kê và học máy, bài toán hồi quy tuyến tính thường yêu cầu tìm tham số $x$ sao cho sai số giữa giá trị thực và giá trị dự đoán là nhỏ nhất, tức là giải bài toán **bình phương tối thiểu**. Phân rã QR là một phương pháp mạnh mẽ để giải bài toán này.

##### **Cách áp dụng**:
Bài toán hồi quy tuyến tính có thể được viết lại dưới dạng $Ax = b$, với $A$ là ma trận đặc trưng của dữ liệu và $b$ là giá trị mục tiêu. Sau khi phân rã $A = QR$, ta chuyển bài toán thành:
$$
\min ||QR x - b||_2 = \min ||R x - Q^T b||_2
$$
Vì $R$ là ma trận tam giác trên, việc giải $R x = Q^T b$ trở nên đơn giản và ổn định.

##### **Ứng dụng thực tế**:
- **Hồi quy tuyến tính**: Sử dụng phân rã QR trong các mô hình hồi quy tuyến tính để giảm thiểu sai số và tăng độ chính xác trong dự đoán.
- **Xử lý dữ liệu**: Giải quyết các vấn đề tối ưu hóa trong phân tích dữ liệu, ví dụ như trong học máy với các bài toán hồi quy.

Phân rã QR giúp giảm thiểu sai số và tăng cường độ ổn định khi giải bài toán bình phương tối thiểu, đặc biệt khi dữ liệu có số lượng lớn hoặc có ma trận đặc trưng có đặc điểm xấu (suy biến).

---

#### **4. Ổn định số và tính toán ma trận**

##### **Mô tả tổng quát**:
Một trong những ưu điểm lớn của phân rã QR là **ổn định số học**, đặc biệt khi đối mặt với các ma trận gần suy biến (ill-conditioned matrices), nơi các phương pháp khác như phân rã LU có thể gặp vấn đề về độ chính xác.

##### **Cách áp dụng**:
Phân rã QR sử dụng các ma trận trực giao, điều này giúp giảm thiểu sai số số học và giúp tính toán chính xác hơn khi ma trận có độ phân giải thấp hoặc các đặc tính không tốt.

##### **Ứng dụng thực tế**:
- **Mô phỏng khí động học**: Trong các bài toán mô phỏng và tính toán trong khoa học kỹ thuật, phân rã QR giúp đảm bảo tính chính xác trong các phép tính phức tạp.
- **Xử lý tín hiệu số**: Các thuật toán yêu cầu độ chính xác cao, ví dụ như trong xử lý tín hiệu radar, sẽ được cải thiện với sự ổn định của phân rã QR.

Phân rã QR có độ ổn định cao hơn khi xử lý các ma trận gần suy biến, giảm thiểu sai số và giúp đảm bảo tính chính xác trong các tính toán khoa học và kỹ thuật.

---

#### **5. Xử lý tín hiệu và nén dữ liệu**

##### **Mô tả tổng quát**:
Phân rã QR cũng được sử dụng trong xử lý tín hiệu, đặc biệt là khi tái tạo tín hiệu từ các cơ sở trực giao hoặc giảm chiều dữ liệu. Phân rã QR kết hợp với các phương pháp khác như SVD (Singular Value Decomposition) có thể hỗ trợ việc **nén dữ liệu**.

##### **Cách áp dụng**:
Kết hợp phân rã QR và SVD (phân tích kỳ dị) có thể giúp giảm bớt kích thước dữ liệu trong các bài toán như nén hình ảnh, âm thanh hay xử lý tín hiệu. Các ma trận trực giao của phân rã QR tạo ra cơ sở giúp phân tích và nén tín hiệu hiệu quả.

##### **Ứng dụng thực tế**:
- **Nén hình ảnh và âm thanh**: Phân rã QR có thể được dùng trong các ứng dụng như JPEG, MP3 để giảm dung lượng dữ liệu mà không làm mất đi chất lượng quá nhiều.
- **Xử lý tín hiệu radar**: Cải thiện hiệu quả trong việc phân tích và tái tạo tín hiệu radar.

Phân rã QR cung cấp một cách mạnh mẽ để phân tích tín hiệu và giảm chiều dữ liệu, từ đó cải thiện khả năng nén mà vẫn giữ lại các đặc trưng quan trọng.

---

### **Kết luận**:
Phân rã QR là một công cụ mạnh mẽ trong toán học và khoa học máy tính, các ứng dụng của phân rã QR không chỉ trong lý thuyết mà còn trong thực tế, giúp giải quyết các bài toán phức tạp với yêu cầu độ chính xác cao.